In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imutils import paths
import random
import cv2
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.python.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [2]:
epoch_num = 50
init_lr = 1e-3
batchsize = 64
class_num = 2
norm_size = 64
depth = 3

label_dir = {
    "SN": 0,
    "PN": 0,
    "SC": 1,
    "PC": 1,
}

In [3]:
conv_base = ResNet50(weights='imagenet',
                    include_top=False,
                    input_shape=(norm_size,norm_size,depth))

In [21]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [4]:
model = Sequential()
conv_base.trainable = False
model.add(conv_base)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(class_num, activation='sigmoid'))

In [5]:
#Loading the data
print("[INFO] loading images...")
data = []
labels = []
imagePaths = sorted(list(paths.list_images("../Images/Training_Set/")))#args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (norm_size, norm_size))
    image = img_to_array(image)
    data.append(image)

    label_str = imagePath.split(os.path.sep)[-2]

    labels.append(label_dir[label_str])

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

[INFO] loading images...


In [6]:
# partition the data into 80% training_total and 20% testing
#partition the training_total data into 75% training data and 25% validation data 
(trainX_t, testX, trainY_t, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)
(trainX, valX, trainY, valY) = train_test_split(trainX_t, trainY_t, test_size=0.25, random_state=42)

#labels = to_categorical(labels, num_classes=CLASS_NUM)
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=class_num)
valY = to_categorical(valY, num_classes=class_num)
testY = to_categorical(testY, num_classes=class_num)


In [7]:
# construct the image generator for data augmentation
train_datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, 
                                   fill_mode="nearest")
train_generator = train_datagen.flow(trainX, trainY, batch_size=batchsize)

In [8]:
val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow(valX, valY, batch_size=batchsize)

In [9]:
opt = Adam(lr=init_lr, decay=init_lr/epoch_num)
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

In [10]:
# train the network
print("[INFO] training network...")
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001) #automatically alter the learning rate
checkpoint = ModelCheckpoint(filepath="0420_binary_test1.h5", monitor='val_accuracy', 
                             verbose=1, save_best_only=True, mode='max', period=1)
H = model.fit_generator(train_generator, steps_per_epoch=len(trainX)//batchsize,
              validation_data=val_generator, validation_steps=len(valX)//batchsize,
              epochs=epoch_num, verbose=1, callbacks=[checkpoint,reduce_lr])

[INFO] training network...
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 28 steps, validate for 9 steps
Epoch 1/50
27/28 [===========================>..] - ETA: 0s - loss: 0.9617 - accuracy: 0.7560
Epoch 00001: val_accuracy improved from -inf to 0.56944, saving model to 0420_binary_test1.h5
28/28 [==============================] - 12s 419ms/step - loss: 0.9465 - accuracy: 0.7569 - val_loss: 0.9073 - val_accuracy: 0.5694
Epoch 2/50
27/28 [===========================>..] - ETA: 0s - loss: 0.4396 - accuracy: 0.8327
Epoch 00002: val_accuracy did not improve from 0.56944
28/28 [==============================] - 10s 341ms/step - loss: 0.4342 - accuracy: 0.8346 - val_loss: 0.9500 - val_accuracy: 0.5694
Epoch 3/50
27/28 [===========================>..] - ETA: 0s - loss: 0.3433 - accuracy: 0.8687
Epoch 00003: val_accuracy did not improve from 0.56944
28/28 [==============================] - 10s 347ms/step 

Epoch 28/50
27/28 [===========================>..] - ETA: 0s - loss: 0.0782 - accuracy: 0.9717
Epoch 00028: val_accuracy did not improve from 0.56944
28/28 [==============================] - 10s 371ms/step - loss: 0.0775 - accuracy: 0.9719 - val_loss: 1.2041 - val_accuracy: 0.5694
Epoch 29/50
27/28 [===========================>..] - ETA: 0s - loss: 0.0989 - accuracy: 0.9656
Epoch 00029: val_accuracy did not improve from 0.56944
28/28 [==============================] - 10s 362ms/step - loss: 0.0990 - accuracy: 0.9654 - val_loss: 1.2720 - val_accuracy: 0.5694
Epoch 30/50
27/28 [===========================>..] - ETA: 0s - loss: 0.0805 - accuracy: 0.9717
Epoch 00030: val_accuracy did not improve from 0.56944
28/28 [==============================] - 11s 390ms/step - loss: 0.0810 - accuracy: 0.9707 - val_loss: 1.4659 - val_accuracy: 0.5694
Epoch 31/50
27/28 [===========================>..] - ETA: 0s - loss: 0.0752 - accuracy: 0.9755
Epoch 00031: val_accuracy did not improve from 0.56944
28/2

In [ ]:






# save the model to disk
print("[INFO] serializing network...")
model.save("crack.model")

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = NUM_EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("ResNet:Training Loss and Accuracy on crack damage level")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Resnet_plot.png")

In [ ]:
model.predict(testX)


In [ ]:
import matplotlib.pyplot as plt
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = NUM_EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
#plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("ResNet:Training Loss and Accuracy on crack damage level")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Resnet_plot.png")
